In [1]:
import pandas as pd
from IPython.display import display
from nltk.corpus import stopwords

from capstone_project import preprocessor

%matplotlib inline

In [3]:
#the current working directory for python is the capstone_project/notebook folder
data = pd.read_pickle("../output/data/train_data.pkl")
data.fillna("", inplace=True)

In [3]:
import spacy
nlp = spacy.load("en")
test = nlp(unicode("I'll do that. This is better."))
for token in test:
    print token, token.lemma, token.lemma_

I 570 i
'll 533 will
do 490 do
that 475 that
. 419 .
This 496 this
is 488 be
better 615 better
. 419 .


In [25]:
x, y = data.iloc[:, 3:-1].values, data.iloc[:, -1].values
print type(y), y[:10]
test = data['is_duplicate'].values
print type(test), test[:10]

<type 'numpy.ndarray'> [1 1 0 0 0 1 1 0 0 0]
<type 'numpy.ndarray'> [1 1 0 0 0 1 1 0 0 0]


In [5]:
#data[data.isnull().any(axis=1)]


In [6]:
#data.add(data[["question1","question2"]].applymap(preprocess.tokenize))

In [21]:
new_data = preprocessor.create_features(data)
display(new_data.head(2))

,q1_len,q2_len,word_share
218104,33,31,0.0
209216,84,85,0.0


In [12]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
'The sun is shining',
'The weather is sweet',
'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp

tfidf = TfidfVectorizer()
#test = tfidf.fit_transform(data["question1"][:10])
X = tfidf.fit_transform(docs)

In [23]:
#print test, "\n", test.toarray()[0]
#print sp.sparse.hstack(docs)

clf = LogisticRegression()
clf.fit(X,y=[0,0,1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
#https://github.com/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/tfidf_scikit-learn.ipynb

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, tokenizer=preprocessor.tokenize)
clf = LogisticRegression()

x, y = data.iloc[:, 3:-1].values, data.iloc[:, -1].values

pipe = Pipeline([('tfidf', tfidf), ('clf', clf)])

pipe.fit(x, y)

TypeError: coercing to Unicode: need string or buffer, numpy.ndarray found